# Notebook Imports

In [70]:
import pandas as pd
import numpy as np

# Constants

In [71]:
VOCAB_SIZE = 2500

TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-feature.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

# Read and Load Features rom txt Files into Numpy Array

In [72]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [73]:
sparse_train_data.shape

(258362, 4)

In [74]:
sparse_train_data

array([[   0,    2,    1,    1],
       [   0,    3,    1,    2],
       [   0,    4,    1,    1],
       ...,
       [5795, 2287,    0,    1],
       [5795, 2367,    0,    1],
       [5795, 2419,    0,    1]])

In [75]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [76]:
sparse_test_data[-5:]

array([[5793, 2353,    0,    2],
       [5793, 2387,    0,    4],
       [5793, 2403,    0,    1],
       [5793, 2429,    0,    3],
       [5793, 2471,    0,    1]])

In [77]:
print('Num of rows in tr file', sparse_train_data.shape[0])
print('Num of rows in test file', sparse_test_data.shape[0])

Num of rows in tr file 258362
Num of rows in test file 117630


In [78]:
print('Num of emails in trainig file', np.unique(sparse_train_data[:, 0]).size)
print('Num of emails in test file', np.unique(sparse_test_data[:, 0]).size)

Num of emails in trainig file 4015
Num of emails in test file 1724


### How to create Empty DataFrame

In [79]:
column_name = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))

In [80]:
column_name[-5:]

[2495, 2496, 2497, 2498, 2499]

In [81]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [82]:
full_train_data = pd.DataFrame(index=index_names, columns=column_name)
full_train_data.fillna(value=0, inplace = True)

In [83]:
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a Full Matrix from a Sparse Matrix

In [84]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    
    column_full_matrix_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, nr_words))
    doc_id_full_matrix_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index = doc_id_full_matrix_names, columns=column_full_matrix_names)
    full_matrix.fillna(value = 0, inplace = True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occur = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occur
    
    full_matrix.set_index('DOC_ID', inplace=True)
    return full_matrix

In [85]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 15.4 s


In [86]:
full_train_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,13,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,3,1,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,3,1,0,1,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [87]:
do = full_train_data[full_train_data.CATEGORY ==1 ].index

In [88]:
do

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,
            ...
            1877, 1880, 1881, 1882, 1884, 1885, 1887, 1889, 1890, 1895],
           dtype='int64', name='DOC_ID', length=1250)

In [89]:
full_train_data[-20:]

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5767,0,0,1,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
5768,0,0,1,0,0,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5770,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5771,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5772,0,0,1,0,0,3,0,3,0,7,...,0,0,1,0,0,0,0,0,0,0
5773,0,0,1,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
5775,0,5,132,26,1,10,0,47,1,41,...,0,0,2,0,0,0,0,0,0,0
5776,0,6,3,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5777,0,2,1,1,1,2,2,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Trainig Naive Base Classifyer Model

## Counting Probability of Spam

In [90]:
full_train_data.CATEGORY.value_counts()

0    2765
1    1250
Name: CATEGORY, dtype: int64

In [91]:
pd.Series(full_train_data['CATEGORY']).value_counts()

0    2765
1    1250
Name: CATEGORY, dtype: int64

In [92]:
type(pd.Series(full_train_data['CATEGORY']).value_counts())

pandas.core.series.Series

In [93]:
prob_spam = pd.Series(full_train_data['CATEGORY']).value_counts()[1]/full_train_data.shape[0]
prob_spam

0.31133250311332505

In [94]:
full_train_data.shape[0]

4015

In [95]:
full_train_data.CATEGORY.size

4015

In [96]:
full_train_data.CATEGORY.sum()

1250

## Total Number of Words / Tokens

In [97]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']

In [98]:
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [99]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4015,)

In [100]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    182
4     43
dtype: int64

In [101]:
total_wc = email_lengths.sum()
total_wc

430991

## Number of Tokens in Spam & Ham Emails

In [102]:
##spam_lenght = email_lengths[full_train_data[full_train_data.CATEGORY ==1 ].index]
spam_lenght = email_lengths[full_train_data.CATEGORY ==1 ]
total_spam_words = spam_lenght.sum()
print('Total words in spam mess', total_spam_words)
spam_lenght

Total words in spam mess 178612


DOC_ID
0        87
1        53
2        40
3       182
4        43
       ... 
1885     45
1887     22
1889    115
1890     26
1895    162
Length: 1250, dtype: int64

In [103]:
spam_lenght = email_lengths[full_train_data[full_train_data.CATEGORY ==1 ].index]
##spam_lenght = full_train_data[full_train_data.CATEGORY ==1 ]
spam_wc = spam_lenght.sum()
print('Total words in spam mess', spam_wc)
spam_lenght

Total words in spam mess 178612


DOC_ID
0        87
1        53
2        40
3       182
4        43
       ... 
1885     45
1887     22
1889    115
1890     26
1895    162
Length: 1250, dtype: int64

In [104]:
##ham_lenght = email_lengths[full_train_data[full_train_data.CATEGORY ==0 ].index]
ham_lenght = email_lengths[full_train_data.CATEGORY == 0]
nonspam_wc = ham_lenght.sum()
print('Total words in ham mess', nonspam_wc)
ham_lenght

Total words in ham mess 252379


DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
       ... 
5789     18
5790     72
5791     45
5794     57
5795    264
Length: 2765, dtype: int64

In [105]:
email_lengths[full_train_data.CATEGORY == 1].index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    9,   10,
            ...
            1877, 1880, 1881, 1882, 1884, 1885, 1887, 1889, 1890, 1895],
           dtype='int64', name='DOC_ID', length=1250)

In [106]:
full_train_data['CATEGORY' == 1]

DOC_ID
0       0
1       7
2       6
3       6
4       5
       ..
5789    3
5790    1
5791    3
5794    1
5795    3
Name: 0, Length: 4015, dtype: int64

In [107]:
ham_lenght.shape[0]

2765

In [108]:
total_wc - spam_wc - nonspam_wc

0

In [109]:
spam_lenght.mean()

142.8896

In [110]:
ham_lenght.mean()

91.27631103074141

## Summing the Tokens Ocurring in Spam

In [111]:
full_train_features.shape

(4015, 2500)

In [112]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,13,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [113]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [114]:
train_spam_tokens.shape

(1250, 2500)

In [115]:
train_ham_tokens.shape

(2765, 2500)

In [116]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens.shape

(2500,)

In [117]:
summed_spam_tokens

0       2177
1        936
2       1220
3       2026
4       1211
        ... 
2495      23
2496      16
2497      11
2498       3
2499      11
Length: 2500, dtype: int64

In [118]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens.shape

(2500,)

In [119]:
summed_ham_tokens

0       5483
1       2588
2       2044
3        938
4       1611
        ... 
2495      12
2496      14
2497      21
2498      31
2499      22
Length: 2500, dtype: int64

In [120]:
train_ham_tokens[2499].sum()+1

22

# P (Token | Spam) - Calculating probability of a Token Occors  given the Email is Spam

In [121]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_ham = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)

In [122]:
prob_tokens_spam

0       0.012020
1       0.005168
2       0.006736
3       0.011186
4       0.006686
          ...   
2495    0.000127
2496    0.000088
2497    0.000061
2498    0.000017
2499    0.000061
Length: 2500, dtype: float64

# P (Token | Ham) - Calculating probability of a Token Occors  given the Email is Ham

In [123]:
prob_tokens_ham

0       0.021512
1       0.010154
2       0.008019
3       0.003680
4       0.006321
          ...   
2495    0.000047
2496    0.000055
2497    0.000082
2498    0.000122
2499    0.000086
Length: 2500, dtype: float64

In [124]:
prob_tokens_spam.sum()

0.9999999999999999

In [125]:
prob_tokens_ham.sum()

1.0

## P(Token) - probability the Token Occurs

In [126]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [127]:
prob_tokens_all

0       0.017768
1       0.008172
2       0.007569
3       0.006873
4       0.006543
          ...   
2495    0.000077
2496    0.000065
2497    0.000070
2498    0.000074
2499    0.000072
Length: 2500, dtype: float64

In [128]:
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [129]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare Test Data

In [130]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [132]:
sparse_test_data.shape

(117630, 4)

In [134]:
##column_name_test = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
##index_names_test = np.unique(sparse_test_data[:, 0])
##full_test_data = pd.DataFrame(index=index_names_test, columns=column_name_test)
##full_test_data.fillna(value=0, inplace = True)

In [142]:
##full_test_data

In [143]:
%%time
full_test_data = make_full_matrix(sparse_test_data, VOCAB_SIZE)

Wall time: 6.5 s


In [145]:
full_test_features = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']

In [146]:
full_test_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,0,0,1,4,2,1,2,4,1,2,...,0,0,0,0,0,0,0,0,0,0
12,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,2,1,1,2,0,0,3,0,4,...,0,0,0,0,0,0,0,0,1,0
17,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,2,1,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,5,5,2,2,1,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,4,0,2,4,3,3,1,4,3,...,0,0,0,0,0,0,0,0,0,0


In [147]:
full_test_targets = full_test_data.loc[:, full_test_data.columns == 'CATEGORY']

In [140]:
full_test_targets

,CATEGORY
DOC_ID,
8,1
12,1
14,1
15,1
17,1
...,...
5783,0
5786,0
5788,0


In [148]:
np.savetxt(TEST_FEATURE_MATRIX, full_test_features)
np.savetxt(TEST_TARGET_FILE, full_test_targets)